In [ ]:
from dotenv import load_dotenv
import json
import os
import random
import requests
from typing import Optional, List
import psycopg2

In [ ]:
from src.utils import OFFERS_START_DATE, OFFERS_END_DATE

### Set Up

In [ ]:
# Load env file
load_dotenv()
X_RAPIDAPI_KEY = os.getenv("X_RAPIDAPI_KEY")
X_RAPIDAPI_HOST = os.getenv("X_RAPIDAPI_HOST")
DB_USER = os.getenv("DB_USER")

In [ ]:
headers = {
	"X-RapidAPI-Key": X_RAPIDAPI_KEY ,
	"X-RapidAPI-Host": X_RAPIDAPI_HOST
}

In [ ]:
def run_query(sql_query, params=None):
    try:
        conn = psycopg2.connect(dbname='offers', user=DB_USER, host='localhost', port='5432')
        cur = conn.cursor()
        
        # Execute the query with parameters safely
        cur.execute(sql_query, params or ())
        
        # For SELECT queries
        if sql_query.strip().upper().startswith("SELECT"):
            result = cur.fetchall()
        else:  # For INSERT/UPDATE/DELETE queries
            conn.commit()
            result = None
        
        cur.close()
        conn.close()
        
        return result
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# sample complex query

url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/complexSearch"

querystring = {"query":"pasta",
    "cuisine":"italian",
    "excludeCuisine":"greek",
    "diet":"vegetarian",
    "intolerances":"gluten",
    "equipment":"pan",
    "includeIngredients":"tomato,cheese",
    "excludeIngredients":"eggs",
    "type":"main course",
    "instructionsRequired":"true",
    "fillIngredients":"false",
    "addRecipeInformation":"false",
    "titleMatch":"Crock Pot",
    "maxReadyTime":"20",
    "ignorePantry":"true",
    "sort":"calories",
    "sortDirection":"asc",
    "minCarbs":"10",
    "maxCarbs":"100",
    "minProtein":"10",
    "maxProtein":"100",
    "minCalories":"50",
    "maxCalories":"800",
    "minFat":"10",
    "maxFat":"100",
    "minAlcohol":"0",
    "maxAlcohol":"100",
    "minCaffeine":"0",
    "maxCaffeine":"100",
    "minCopper":"0",
    "maxCopper":"100",
    "minCalcium":"0",
    "maxCalcium":"100",
    "minCholine":"0",
    "maxCholine":"100",
    "minCholesterol":"0",
    "maxCholesterol":"100",
    "minFluoride":"0",
    "maxFluoride":"100",
    "minSaturatedFat":"0",
    "maxSaturatedFat":"100",
    "minVitaminA":"0",
    "maxVitaminA":"100",
    "minVitaminC":"0",
    "maxVitaminC":"100",
    "minVitaminD":"0",
    "maxVitaminD":"100",
    "minVitaminE":"0",
    "maxVitaminE":"100",
    "minVitaminK":"0",
    "maxVitaminK":"100",
    "minVitaminB1":"0",
    "maxVitaminB1":"100",
    "minVitaminB2":"0",
    "maxVitaminB2":"100",
    "minVitaminB5":"0",
    "maxVitaminB5":"100",
    "minVitaminB3":"0",
    "maxVitaminB3":"100",
    "minVitaminB6":"0",
    "maxVitaminB6":"100",
    "minVitaminB12":"0",
    "maxVitaminB12":"100",
    "minFiber":"0",
    "maxFiber":"100",
    "minFolate":"0",
    "maxFolate":"100",
    "minFolicAcid":"0",
    "maxFolicAcid":"100",
    "minIodine":"0",
    "maxIodine":"100",
    "minIron":"0",
    "maxIron":"100",
    "minMagnesium":"0",
    "maxMagnesium":"100",
    "minManganese":"0",
    "maxManganese":"100",
    "minPhosphorus":"0",
    "maxPhosphorus":"100",
    "minPotassium":"0",
    "maxPotassium":"100",
    "minSelenium":"0",
    "maxSelenium":"100",
    "minSodium":"0",
    "maxSodium":"100",
    "minSugar":"0",
    "maxSugar":"100",
    "minZinc":"0",
    "maxZinc":"100",
    "offset":"0",
    "number":"10",
    "limitLicense":"false",
    "ranking":"2"
    }


In [ ]:
food_url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/complexSearch"


def find_recipe(
        query: str,
        diet: Optional[str] = None,
        include_ingredients: Optional[List[str]] = None,  # Assuming this is a list of ingredient names
        exclude_ingredients: Optional[List[str]] = None,
        type: str = "main course",
        instructions_required: bool = True,
        fill_ingredients: bool = False,
        add_recipe_information: bool = False,
        cuisine: Optional[str] = None,
        exclude_cuisine: Optional[str] = None,
        max_ready_time: Optional[int] = None,
        ignore_pantry: bool = True,
        min_carbs: Optional[int] = None,
        max_carbs: Optional[int] = None,
        min_protein: Optional[int] = None,
        max_protein: Optional[int] = None,
        min_calories: Optional[int] = None,
        max_calories: Optional[int] = None,
        number: int = 2
        ):
    
    querystring = {
        "query": query,
        "type": type,
        "number": number
    }
    
    # Add optional parameters only if they are not None
    if diet is not None:
        querystring["diet"] = diet
    if include_ingredients is not None:
        querystring["includeIngredients"] = ",".join(include_ingredients)
    if exclude_ingredients is not None:
        querystring["excludeIngredients"] = ",".join(exclude_ingredients)
    if instructions_required is not None:
        querystring["instructionsRequired"] = str(instructions_required).lower()
    if fill_ingredients is not None:
        querystring["fillIngredients"] = str(fill_ingredients).lower()
    if cuisine is not None:
        querystring["cuisine"] = cuisine
    if exclude_cuisine is not None:
        querystring["excludeCuisine"] = exclude_cuisine
    if max_ready_time is not None:
        querystring["maxReadyTime"] = max_ready_time
    if add_recipe_information is not None:
        querystring["addRecipeInformation"] = str(add_recipe_information).lower(),
    if ignore_pantry is not None:
        querystring["ignorePantry"] = str(ignore_pantry).lower(),
    if min_carbs is not None:
        querystring["minCarbs"] = min_carbs
    if max_carbs is not None:
        querystring["maxCarbs"] = max_carbs
    if min_protein is not None:
        querystring["minProtein"] = min_protein
    if max_protein is not None:
        querystring["maxProtein"] = max_protein
    if min_calories is not None:
        querystring["minCalories"] = min_calories
    if max_calories is not None:
        querystring["maxCalories"] = max_calories
        

    response = requests.get(food_url, headers=headers, params=querystring)
    return response.json()


In [ ]:
def get_recipe_info(recipe_id: int):
    url = f"https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/{recipe_id}/information"
    response = requests.get(url, headers=headers)
    return response.json()

### Options

In [ ]:
cuisine_options = [
    "African",
    "Asian",
    "American",
    "British",
    "Cajun",
    "Caribbean",
    "Chinese",
    "Eastern European",
    "European",
    "French",
    "German",
    "Greek",
    "Indian",
    "Irish",
    "Italian",
    "Japanese",
    "Jewish",
    "Korean",
    "Latin American",
    "Mediterranean",
    "Mexican",
    "Middle Eastern",
    "Nordic",
    "Southern",
    "Spanish",
    "Thai",
    "Vietnamese"
]



In [ ]:
cuisine_options

In [ ]:
string = 'African', 'Asian', 'American', 'British', 'Cajun', 'Caribbean', 'Chinese', 'Eastern European', 'European', 'French', 'German', 'Greek', 'Indian', 'Irish', 'Italian', 'Japanese', 'Jewish', 'Korean', 'Latin American', 'Mediterranean', 'Mexican', 'Middle Eastern', 'Nordic', 'Southern', 'Spanish', 'Thai', 'Vietnamese'


In [ ]:
diet_options = [
    "vegetarian",
    "lacto vegetarian",
    "ovo vegetarian",
    "vegan",
    "pescetarian",
    "paleo",
    "primal"
]

In [ ]:
intolerances_options = [
    "dairy",
    "egg",
    "gluten", 
    "peanut", 
    "sesame", 
    "seafood", 
    "shellfish", 
    "soy", 
    "sulfite", 
    "tree nut", 
    "wheat"
    ]

In [ ]:
type_options = [
    "main course",
    "side dish",
    "dessert",
    "appetizer",
    "salad",
    "bread",
    "breakfast",
    "soup",
    "beverage",
    "sauce",
    "marinade",
    "fingerfood",
    "snack",
    "drink"
]

In [ ]:
max_ready_time_options = [
    20, 30, 45, 60, 90, 120, None
]

In [ ]:
dish_name_options = [
    "a pasta",
    "a pizza",
    "burgers",
    "a salad",
    "a soup",
    "rice dish",
    "a sandwich",
    "a snack",
    "a quinoa dish",
    "a lentil based dish",
    "a smoothie",
    # 
]

In [ ]:
# TODO: needs to be replaced by list from database!!!
# TODO: discuss translations

ingredients_options = [
    "strawberries",
    "broccoli",
    "carrots",
    "potatoes",
    "tomatoes",
    "onions",
    "garlic",
    "ginger",
    "parsley",
    "peanuts",
    "spinach",
    "mushrooms"
]

## Create the Prompt for Recipes

In [ ]:
def format_list(items, action_word, include_type=True, item_type="cuisine"):
    if len(items) > 1:
        formatted_string = ", ".join(items[:-1]) + ", and " + items[-1]
        type_plural = f"{item_type}s" if include_type else ""
        return f"I {action_word} {formatted_string} {type_plural}."
    elif len(items) == 1:
        type_singular = f"{item_type}" if include_type else ""
        return f"I {action_word} {items[0]} {type_singular}."
    return ""

def preferences(number_cuisine_include, number_cuisine_exclude, cuisine_options, intolerances=[]):
    all_cuisines = set(cuisine_options)
    
    cuisine_include = set(random.sample(cuisine_options, min(number_cuisine_include, len(cuisine_options))))
    remaining_cuisines = all_cuisines - cuisine_include
    cuisine_exclude = set(random.sample(list(remaining_cuisines), min(number_cuisine_exclude, len(remaining_cuisines))))

    # For preferences
    like_string = format_list(list(cuisine_include), "prefer")
    dislike_string = format_list(list(cuisine_exclude), "dislike")

    # # For intolerances
    # intolerance_string = format_list(intolerances, "am intolerant to", include_type=False)

    return like_string, dislike_string

def intolerances(intolerances_options, number_intolerances):
    intolerances = set(random.sample(intolerances_options, number_intolerances))
    intolerance_string = format_list(list(intolerances), "am intolerant to", include_type=False)
    return intolerance_string

# testing:
number_cuisine_include = 3
number_cuisine_exclude = 2
like_string, dislike_string= preferences(number_cuisine_include, number_cuisine_exclude, cuisine_options)
print(like_string)
print(dislike_string)

intolerance_string = intolerances(intolerances_options, 4)
print(intolerance_string)


In [ ]:
def generate_cooking_time_phrase(max_time):
    # If None, return an empty string
    if max_time is None:
        return ""
    
    # Generate phrases based on the selected max time
    phrases = [
        f"I want a {max_time}-min recipe.",
        f"I want to spend a maximum of {int(max_time/60)} hours cooking." if max_time >= 60 else f"I want to spend a maximum of {max_time} minutes cooking.",
        f"The recipe should take me no longer than {max_time} min."
    ]
    
    # Select a phrase randomly to return
    return random.choice(phrases)

# Example usage
max_time = random.choice(max_ready_time_options)
print(generate_cooking_time_phrase(max_time))


In [ ]:
def create_prompt():
    # select a random dish from dish_name_options:
    dish_name = random.choice(dish_name_options)
    question_recipe = f"I would like to make {dish_name}."

    # select cuisine options:
    number_cuisine_include = random.randint(1, 3) # alternative: randint(1, len(cuisine_options)) 
    number_cuisine_exclude = random.randint(0, 2) # alternative: randint(0, len(cuisine_options))
    like_string, dislike_string = preferences(number_cuisine_include, number_cuisine_exclude, cuisine_options)
    
    # select dietary restrictions:
    diet = random.choice(diet_options)
    diet_string = f"I have a {diet} diet."

    # select intolerances:
    number_intolerances = random.randint(0, len(intolerances_options))
    intolerance_string = intolerances(intolerances_options, number_intolerances)

    # select a random cooking time:
    max_time = random.choice(max_ready_time_options)
    cooking_time = generate_cooking_time_phrase(max_time)

    # Group related components
    core_group = [question_recipe, diet_string]
    preference_group = [like_string, dislike_string, intolerance_string]
    
    # Shuffle the groups, but keep the group's internal order
    groups = [core_group, preference_group, [cooking_time]]
    random.shuffle(groups)

    # Flatten the list of groups and join the components to form the prompt
    prompt = " ".join([item for group in groups for item in group if item])

    # Additionally (for now!) print the parameters to test the API:
    print("#############################################")
    print(f"query = {dish_name}")
    print(f"diet = {diet}")
    print(f"max_ready_time = {max_time}")
    print(f"include_ingredients = {ingredients_options}")
    print(f"cuisine_include = {like_string}")
    print(f"cuisine_exclude = {dislike_string}")
    print("#############################################")
    print("")

    return prompt


In [ ]:
def add_wine_suggestion(prompt):
    # List of sentences to choose from
    wine_suggestions = [
        "Please give me a dish recommendation and a wine suggestion that goes well with that dish.",
        "Additionally to the recipe, could you recommend a wine pairing for this dish?",
        "Please find me a recipe. Also, I'd love a wine recommendation to accompany that dish.",
        "Please find a recipe and suggest a wine that pairs perfectly with this meal, please."
    ]
    
    # Decide randomly whether to add a wine suggestion or not
    if random.choice([True, False]):
        # If true, randomly pick a sentence from the list
        wine_sentence = random.choice(wine_suggestions)
        prompt += f" {wine_sentence}"
    
    return prompt


In [ ]:
# Example testing:
food_prompt = create_prompt()
prompt = add_wine_suggestion(food_prompt)
print(prompt)
print("")

In [ ]:
def compare_to_fridge_and_add_to_shopping_list(prompt):
    # List of sentences to choose from
    fridge_comparison = [
        "Could you check if I have any of the ingredients already at home? Please add all missing ingredients to my shopping list.",
        "Please compare the ingredients to what I have at home and add any missing items to my shopping list.",
        "I'd like to know which ingredients I already have at home. Please add any missing items to my shopping list.",
        "Please check if I have any of the ingredients at home. If not, add all missing items to my shopping list."
    ]
    fridge_sentence = random.choice(fridge_comparison) # Randomly pick a sentence from the list
    prompt += f" {fridge_sentence}" # Append the sentence to the prompt
    
    return prompt

In [ ]:
def prompt_dish_identification():
    """ Prompt to ask for dish identification and recipe based on an image."""

    image_url_list = [
        "https://image.essen-und-trinken.de/11920200/t/Wo/v9/w960/r1/-/spaghetti-bolognese-2a2fc84c1def03b0f1f01e318ad1e067-fjt2014030361-jpg--7759-.jpg", # Spagehtti Bolognese
        "https://image.essen-und-trinken.de/11831462/t/jt/v9/w1440/r1/-/tomaten-focaccia-a0cbd0bf23ea035a2d88fecc760104e0-fjt2014090351-jpg--8327-.jpg", # Tomaten-Focaccia
        "https://image.essen-und-trinken.de/13794116/t/sl/v1/w960/r1.5/-/eut202205093-samosas-mit-kartoffelfuellung-und-gurkensalat.jpg", # Samosas (with cucumber salad)
        "https://image.essen-und-trinken.de/11941600/t/Tz/v10/w1440/r1/-/spargelsuppe-klassisch-jpg--59493-.jpg", # Asparagus Soup
        "https://image.essen-und-trinken.de/11920454/t/CA/v8/w1440/r1/-/pargel-mit-gekochtem-schinken-und-neuen-kartoffeln-aeaf985292c54244bc463951383311d8-et2014050161-jpg--7886-.jpg" # Asparagus with cooked ham and potatoes
    ]

    image_url = random.choice(image_url_list) # Randomly select an image URL
    
    url_prompt = [
        f"Using the image at '{image_url}', ",
        f"Given the image at '{image_url}', ",
        f"Based on the image at '{image_url}', ",
        f"{image_url}, "
                  ]
    # List of sentences to choose from
    dish_classification = [
        "could you identify the dish in the image and provide me with a recipe?",
        "please identify the dish and suggest a recipe.",
        "I'd like to know what dish is in the image and get a recipe for it.",
        "please provide me with a recipe for the dish in the image."
    ]
    
    url_phrase = random.choice(url_prompt) # Randomly select a phrase with the image URL
    classification_phrase = random.choice(dish_classification) # Randomly select a phrase for dish classification
    prompt= f"{url_phrase}{classification_phrase}" # Combine the two phrases to a single prompt
    
    return prompt

In [ ]:
classification_prompt = prompt_dish_identification()
# prompt = compare_to_fridge_and_add_to_shopping_list(classification_prompt)
prompt = add_wine_suggestion(classification_prompt)
print("")
print(prompt)
print("")

### Wine Questions

In [ ]:
# using wine database (regular sortiment, no discounts)
def generate_question_and_sql():
    # Randomly choose the question parameters
    number_of_wines = random.choice([3, 5, 10])
    wine_type = random.choice(['red', 'rose', 'white'])
    price_threshold = random.choice([2, 3, 4, 5])
    weeks_ago = random.choice([1, 2, 3])

    # Map question templates to SQL templates with placeholders for psycopg2
    templates = [
        (f"List the top {number_of_wines} {wine_type} wines sorted by price in ascending order.",
         "SELECT * FROM offers_wine WHERE wine_type = %s ORDER BY price_regular ASC LIMIT %s", (wine_type, number_of_wines)),

        (f"Which {number_of_wines} {wine_type} wines are the most expensive?",
         "SELECT * FROM offers_wine WHERE wine_type = %s ORDER BY price_regular DESC LIMIT %s", (wine_type, number_of_wines)),

        (f"Find the {number_of_wines} least expensive {wine_type} wines in our collection.",
         "SELECT * FROM offers_wine WHERE wine_type = %s ORDER BY price_regular ASC LIMIT %s", (wine_type, number_of_wines)),

        (f"How many {wine_type} wines have a price lower than {price_threshold} Euros?",
         "SELECT COUNT(*) FROM offers_wine WHERE wine_type = %s AND price_regular < %s", (wine_type, price_threshold)),

        (f"What is the average price of the top {number_of_wines} cheapest {wine_type} wines?",
         "SELECT AVG(price_regular) FROM (SELECT price_regular FROM offers_wine WHERE wine_type = %s ORDER BY price_regular ASC LIMIT %s) AS subquery", (wine_type, number_of_wines)),

        (f"List all the {wine_type} wines that have offers starting from the past {weeks_ago} weeks.",
         "SELECT * FROM offers_wine WHERE wine_type = %s AND offer_from >= current_date - INTERVAL '%s weeks'", (wine_type, weeks_ago))
    ]

    # Randomly select a question and its corresponding SQL template
    question, sql_template, params = random.choice(templates)

    return question, sql_template, params


In [ ]:
# Example usage:
question, sql_query, params = generate_question_and_sql()
print("Question:", question)
print("SQL Query:", sql_query)
print("Parameters:", params)

# Directly pass params without additional parentheses
results = run_query(sql_query, params)  

if results:
    for result in results:
        print(result)

In [ ]:
# using wine offers
discount_criteria_mapping = {
    "10% or more": ">= 0.10",
    "at least 20%": ">= 0.20",
    "up to 30%": "<= 0.30",
    "over 5%": "> 0.05",
    "below 15%": "< 0.15"
}

def generate_discount_question_sql():
    criteria_text, criteria_sql = random.choice(list(discount_criteria_mapping.items()))
    wine_type = random.choice(['red', 'rose', 'white'])
    number_of_wines = random.choice([3, 5, 10])
    week_option = random.choice([1, 2, 3, 4])  # Assuming this refers to the number of weeks in the past from the current date

    questions_sql_map = [
        (f"Which {number_of_wines} {wine_type} wines have a discount {criteria_text}?",
         f"SELECT * FROM offers_wine WHERE wine_type = %s AND (price_regular - price_offer) / price_regular {criteria_sql} ORDER BY (price_regular - price_offer) / price_regular DESC LIMIT %s;",
         (wine_type, number_of_wines)),

        (f"List all {wine_type} wines with a discount {criteria_text}.",
         f"SELECT * FROM offers_wine WHERE wine_type = %s AND (price_regular - price_offer) / price_regular {criteria_sql};",
         (wine_type,)),

        (f"How many {wine_type} wines are discounted by {criteria_text}?",
         f"SELECT COUNT(*) FROM offers_wine WHERE wine_type = %s AND (price_regular - price_offer) / price_regular {criteria_sql};",
         (wine_type,)),

        (f"Show the {number_of_wines} {wine_type} wines with the highest relative discounts.",
         f"SELECT * FROM offers_wine WHERE wine_type = %s ORDER BY (price_regular - price_offer) / price_regular DESC LIMIT %s;",
         (wine_type, number_of_wines)),

        (f"Find the average discount applied to {wine_type} wines in the past {week_option} weeks.",
         f"SELECT AVG(price_regular - price_offer) FROM offers_wine WHERE wine_type = %s AND offer_from >= current_date - INTERVAL '%s weeks';",
         (wine_type, week_option))
    ]

    # Randomly select one of the question and SQL template pairs
    selected_question, selected_sql_template, params = random.choice(questions_sql_map)

    # Return the selected question, SQL template, and the accurately matched parameters
    return selected_question, selected_sql_template, params



In [ ]:
# Example usage:
question, sql_query, params = generate_discount_question_sql()
print("Question:", question)
print("SQL Query:", sql_query)
print("Parameters:", params)


# Directly pass params without additional parentheses
results = run_query(sql_query, params)  

if results:
    for result in results:
        print(result)